# Imports

In [8]:
import numpy as np
import pandas as pd
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

print(os.listdir("/home/mariohn/Documentos/VSCODE/Datasets/KDEF+"))



from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.datasets import make_classification
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━ 259.9/585.9 MB 280.2 kB/s eta 0:19:24

# Leitura dos dados

In [ ]:
data_path = '/home/mariohn/Documentos/VSCODE/Datasets/KDEF+'
data_dir_list = os.listdir(data_path)

num_epoch=10

img_data_list=[]


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(300,300))
        img_data_list.append(input_img_resize)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

In [ ]:
cv2.imshow(img_data[0], cmap='gray')
plt.show()

# Pipeline

# LL transform

In [ ]:
def log_img(img):
    c = 255/np.log(1+np.max(img))
    log_image = c*(np.log(img+1))
    log_image = np.array(log_image, dtype=np.uint8)
    return log_image

In [ ]:
def laplacian_transformation(img):
    blur = cv2.GaussianBlur(img,(3,3),0)
    laplacian = cv2.Laplacian(blur, cv2.CV_64F)
    return laplacian

In [ ]:
def LLtransform(data):
    img_ls = []
    for img in data:
        img_new = log_img(img)
        img_new = laplacian_transformation(img_new)
        img_ls.append(img_new)
    new_data = np.array(img_ls)
    # new_data = new_data.astype('float32')
    # new_data = new_data/255
    return new_data

In [ ]:
data pipelined = LLtransform()

# Rotulando e dividindo em treino e teste

In [ ]:
num_classes = 7

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:134]=0 #135
labels[135:188]=1 #54
labels[189:365]=2 #177
labels[366:440]=3 #75
labels[441:647]=4 #207
labels[648:731]=5 #84
labels[732:980]=6 #249

names = ['anger','contempt','disgust','fear','happy','sadness','surprise']

def getLabel(id):
    return ['anger','contempt','disgust','fear','happy','sadness','surprise'][id]

In [ ]:
Y = np_utils.to_categorical(labels, num_classes)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42, stratify=Y)

# TFP

# Classificação